In [10]:
# AIM: 7 - ASSESS DATA
# 01-Vermeulen - v1)To illustrate the consolidation process
# MSCIT PART 1 - SEM 1 
# ROHAN PALKAR - 05-DEC-2022
# UNIVERSITY DEPARTMENT OF INFORMATION TECHNOLOGY

In [11]:
import sys 
import os
from datetime import datetime 
from pytz import timezone 
import pandas as pd
import sqlite3 as sq 
import uuid

In [12]:
pd.options.mode.chained_assignment = None 
################################################################ 
InputFileName='C:/VKHCG/01-Vermeulen/00-RawData/VehicleData.csv'
################################################################
sDataBaseDir='C:/VKHCG/01-Vermeulen/04-Transform/SQLite' 
if not os.path.exists(sDataBaseDir):
    os.makedirs(sDataBaseDir) 
################################################################
sDatabaseName=sDataBaseDir + '/Vermeulen.db' 
conn1 = sq.connect(sDatabaseName)
################################################################
sDataVaultDir='C:/VKHCG/88-DV'
if not os.path.exists(sDataVaultDir): 
    os.makedirs(sDataVaultDir)
################################################################
sDatabaseName=sDataVaultDir + '/datavault.db' 
conn2 = sq.connect(sDatabaseName)

In [13]:
sDataWarehouseDir='C:/VKHCG/99-DW'
if not os.path.exists(sDataWarehouseDir): 
    os.makedirs(sDataWarehouseDir)
################################################################
sDatabaseName=sDataWarehouseDir + '/datawarehouse.db' 
conn3 = sq.connect(sDatabaseName)
################################################################ 
print('\n#################################')
print('Time Category') 
print('UTC Time')
BirthDateUTC = datetime(1960,12,20,10,15,0) 
BirthDateZoneUTC=BirthDateUTC.replace(tzinfo=timezone('UTC')) 
BirthDateZoneStr=BirthDateZoneUTC.strftime("%Y-%m-%d %H:%M:%S") 
BirthDateZoneUTCStr=BirthDateZoneUTC.strftime("%Y-%m-%d %H:%M:%S (%Z) (%z)") 
print(BirthDateZoneUTCStr)
print('#################################')
print('Birth Date in Reykjavik :') 
BirthZone = 'Atlantic/Reykjavik'
BirthDate = BirthDateZoneUTC.astimezone(timezone(BirthZone)) 
BirthDateStr=BirthDate.strftime("%Y-%m-%d %H:%M:%S (%Z) (%z)") 
BirthDateLocal=BirthDate.strftime("%Y-%m-%d %H:%M:%S") 
print(BirthDateStr)
print('#################################') 


#################################
Time Category
UTC Time
1960-12-20 10:15:00 (UTC) (+0000)
#################################
Birth Date in Reykjavik :
1960-12-20 09:15:00 (-01) (-0100)
#################################


In [14]:
################################################################
IDZoneNumber=str(uuid.uuid4()) 
sDateTimeKey=BirthDateZoneStr.replace(' ','-').replace(':','-') 
TimeLine=[('ZoneBaseKey', ['UTC']),
    ('IDNumber', [IDZoneNumber]), 
    ('DateTimeKey', [sDateTimeKey]), 
    ('UTCDateTimeValue', [BirthDateZoneUTC]), 
    ('Zone', [BirthZone]),
    ('DateTimeValue', [BirthDateStr])] 
TimeFrame = pd.DataFrame.from_items(TimeLine)
################################################################
TimeHub=TimeFrame[['IDNumber','ZoneBaseKey','DateTimeKey','DateTimeValue']] 
TimeHubIndex=TimeHub.set_index(['IDNumber'],inplace=False) 
################################################################
sTable = 'Hub-Time-Gunnarsson' 
print('\n#################################')
print('Storing :',sDatabaseName,'\n Table:',sTable) 
print('\n#################################')


#################################
Storing : C:/VKHCG/99-DW/datawarehouse.db 
 Table: Hub-Time-Gunnarsson

#################################


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  # Remove the CWD from sys.path while we load stuff.


In [15]:
TimeHubIndex.to_sql(sTable, conn2, if_exists="replace") 
sTable = 'Dim-Time-Gunnarsson' 

In [16]:
TimeHubIndex.to_sql(sTable, conn3, if_exists="replace")

In [17]:
################################################################
TimeSatellite=TimeFrame[['IDNumber','DateTimeKey','Zone','DateTimeValue']] 
TimeSatelliteIndex=TimeSatellite.set_index(['IDNumber'],inplace=False) 
################################################################
BirthZoneFix=BirthZone.replace(' ','-').replace('/','-') 
sTable = 'Satellite-Time-' + BirthZoneFix + '-Gunnarsson' 
print('\n#################################')
print('Storing :',sDatabaseName,'\n Table:',sTable)


#################################
Storing : C:/VKHCG/99-DW/datawarehouse.db 
 Table: Satellite-Time-Atlantic-Reykjavik-Gunnarsson


In [18]:
print('\n#################################')
TimeSatelliteIndex.to_sql(sTable, conn2, if_exists="replace") 
sTable = 'Dim-Time-' + BirthZoneFix + '-Gunnarsson' 
TimeSatelliteIndex.to_sql(sTable, conn3, if_exists="replace")
################################################################
print('\n#################################')
print('Person Category') 
FirstName = 'Guðmundur' 
LastName = 'Gunnarsson'
print('Name:',FirstName,LastName) 
print('Birth Date:',BirthDateLocal) 
print('Birth Zone:',BirthZone)
print('UTC Birth Date:',BirthDateZoneStr) 
print('#################################') 
###############################################################
IDPersonNumber=str(uuid.uuid4()) 
PersonLine=[('IDNumber', [IDPersonNumber]),
    ('FirstName', [FirstName]),
    ('LastName', [LastName]),
    ('Zone', ['UTC']),
    ('DateTimeValue', [BirthDateZoneStr])] 
PersonFrame = pd.DataFrame.from_items(PersonLine)
################################################################
TimeHub=PersonFrame 
TimeHubIndex=TimeHub.set_index(['IDNumber'],inplace=False) 
################################################################
sTable = 'Hub-Person-Gunnarsson' 
print('\n#################################')
print('Storing :',sDatabaseName,'\n Table:',sTable) 
print('\n#################################')
TimeHubIndex.to_sql(sTable, conn2, if_exists="replace") 
sTable = 'Dim-Person-Gunnarsson' 
TimeHubIndex.to_sql(sTable, conn3, if_exists="replace")
################################################################



#################################

#################################
Person Category
Name: Guðmundur Gunnarsson
Birth Date: 1960-12-20 09:15:00
Birth Zone: Atlantic/Reykjavik
UTC Birth Date: 1960-12-20 10:15:00
#################################

#################################
Storing : C:/VKHCG/99-DW/datawarehouse.db 
 Table: Hub-Person-Gunnarsson

#################################


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
